In [1]:
using BSON: @save, @load
using Statistics
using DelimitedFiles
using Printf
using MDToolbox
using StatsBase 
using LinearAlgebra
using Random
using Plots

include("../src/afm.jl")

baumwelch (generic function with 1 method)

In [2]:
@load "data/viterbi/test_radius_$(25)/pred_radius_$(18)/sharpness_$(10)/q_$(576)/T__baumwelch.bson" T learned_T

In [3]:
learned_T

1140×1140 Matrix{Float64}:
 0.0           0.0           0.0           …  0.0          0.0
 0.0           4.29464e-174  5.25357e-15      0.0          0.0
 0.0           1.00505e-15   0.0              0.0          0.0
 0.0           1.12672e-20   0.0              0.0          0.0
 2.2228e-241   0.0           2.68191e-273     0.0          0.0
 0.0159099     8.45664e-115  0.105501      …  0.0          0.0
 0.0           3.36869e-96   0.0              0.0          0.0
 0.0           0.0           0.0              0.0          0.0
 0.0           3.64746e-247  0.0              0.0          0.0
 0.0           0.112364      3.27031e-293     0.0          0.0
 6.86013e-308  6.92025e-32   2.62114e-52   …  0.0          0.0
 1.37677e-318  1.23383e-119  7.05861e-89      0.0          0.0
 0.126444      0.0213195     3.92582e-100     0.0          0.0
 ⋮                                         ⋱               
 0.0           0.0           0.0              0.0          0.0
 0.0           0.0           0.

In [4]:
@load "data/viterbi/test_radius_$(25)/pred_radius_$(18)/sharpness_$(10)/q_$(576)/rmsds_baumwelch.bson" rmsds_arr

In [5]:
mean(rmsds_arr[extracted_qs, :])

LoadError: UndefVarError: extracted_qs not defined

In [6]:
using DataFrames

nq = 576
test_radius = 25
pred_radii = [15, 18, 20, 25, 30, 32, 35]
#pred_radii_str = ["1.8", "2.0", "2.5", "3.0", "3.2"]
labels = ["1 frame-by-frame rigid-body fitting", "2 Viterbi with the ground-truth T", "3 Viterbi with estimated T", "4 Viterbi with random matrix"]
sigma_noise = 3
sharpness = 10
nframe = 100

plts = []
x_min = 0
x_max = 70
bin_length = 30
font_size = 10
width = (x_max - x_min) / bin_length

seed = MersenneTwister(334)
extracted_qs = sample(seed, 1:576, 50, replace=false)

rmsds = []
for r in pred_radii
    @load "data/viterbi/test_radius_$(test_radius)/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_origin.bson" rmsds_arr
    @show size(rmsds_arr)
    x = DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label= labels[1])
    @load "data/viterbi/test_radius_$(test_radius)/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_MD.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[2]))
    @load "data/viterbi/test_radius_$(test_radius)/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_baumwelch.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[3]))
    @load "data/viterbi/test_radius_$(test_radius)/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_rand.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[4]))
    
    push!(rmsds, x)
end

#for i = 2:length(pred_radii)
#DataFrame
#end

size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)


In [7]:
df = rmsds[4]
df = filter(:label => ==("1 frame-by-frame rigid-body fitting"), df)
sum(df.rmsd .== 0.0)

4991

In [8]:
df = rmsds[4]
df = filter(:label => ==("2 Viterbi with the ground-truth T"), df)
sum(df.rmsd .== 0.0)

4991

In [9]:
df = rmsds[4]
df = filter(:label => ==("3 Viterbi with estimated T"), df)
sum(df.rmsd .== 0.0)

4991

In [10]:
rmsds[1].label

20000-element Vector{String}:
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 ⋮
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"

In [11]:
using StatsPlots
ps = []

for i in 1:length(pred_radii)
    p = @df rmsds[i] groupedhist(:rmsd, group=:label, bar_position=:dodge, 
    bar_width=0.4, bins=20, framestyle=:box, size=(1100, 250), dpi=300, 
    xlim=(-0.3, 8), #ylim=(0, 3000), 
    #labels=map(x -> x[3:end], labels), 
    label=nothing, 
    xlabel="RMSD [nm]", ylabel="frequency", 
    legend=:top, 
    left_margin=Plots.Measures.Length(:mm, 10.0),
    bottom_margin=Plots.Measures.Length(:mm, 10.0))
    push!(ps, p)
end  

In [86]:
for i in 1:length(ps)
    savefig(ps[i], "fig03_$(pred_radii[i])A.png")
end

In [12]:
for i in 1:length(ps)
    savefig(ps[i], "fig03_$(pred_radii[i])A_hogehoge.png")
end